In [21]:
import os, re, csv
import pandas as pd
import numpy as np
from kiwipiepy import Kiwi

In [28]:
# 데이터셋 불러와서 정리하기

eta_df = pd.read_csv(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\질문.csv", encoding='ANSI')
eta_df = eta_df.drop(['Unnamed: 0', '글쓴이', '더보기', 'id', '작성시간', '좋아요', '댓글 수'], axis=1)
eta_df['TAGSET'] = np.nan
eta_df.rename(columns={"내용":"QUESTION", "댓글":"ANSWER"}, inplace=True)

eta2_df = pd.read_csv(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\질문2.csv", encoding='ANSI')
eta2_df = eta2_df.drop(['Unnamed: 0', '댓글쓴이', '대댓글여부', 'id'], axis=1)
eta2_df['TAGSET'] = np.nan
eta2_df.rename(columns={"내용":"QUESTION", "댓글":"ANSWER"}, inplace=True)

dm_df = pd.read_csv(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\이중전과게시판_이중검색.csv", encoding='UTF-8')
dm_df = dm_df.drop(['Unnamed: 0', '댓글쓴이', '대댓글여부', 'id'], axis=1)
dm_df['TAGSET'] = np.nan
dm_df.rename(columns={"내용":"QUESTION", "댓글":"ANSWER"}, inplace=True)

# 데이터 병합
df_lst = [eta_df, eta2_df, dm_df]
merged_df = pd.concat(df_lst)
merged_df.reset_index(drop=True, inplace=True)
merged_df = merged_df[['QUESTION', "ANSWER", "TAGSET"]]

,QUESTION,ANSWER,TAGSET
0,조교님! 이중전공이랑 부전공 차이에 대해서 궁금해요!이중전공은 학위+졸업장 표시인건...,NaN,NaN
1,안녕하세요 조교님!재수강 연결방밥 대로 종합정보시스템-수업수강관리-재수강과목선택메뉴...,NaN,NaN
2,조교님 안녕하세요!! 항상 수고가 많으십니다 다름아니라 제가 총평점이 좀 낮은편이라...,NaN,NaN
3,안녕하세요 조교님!제가 맨 처음에 이중전공을 gbt로 신청했다가이중전공 경영으로 변...,NaN,NaN
4,이중전공을 바꾸고싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요? 아...,NaN,NaN
...,...,...,...
6885,안녕하세요 중외통 17학번인데요이중전공 어떤 걸로 하면 좋은지 추천해주세요 ㅎㅎ,['아 그렇군요! 감사합니다!!'],NaN
6886,공대생 이중전공으로 언어할까 하는데 힘듦? 언어추천좀,['스페인어 했다가 힘들어서 영어로 바꾼 사람은 봤음'],NaN
6887,국제통상학과 이중전공생인데 걱정이 많이 된다..,"['저도요.... 진짜 국통법 때문에 미칠거 같네요 ,', '나두...', '국통 ...",NaN
6888,경영 후기 이중전공 신청할 때이중전공 떨어져서 부전공 경영인데후기 이중 경영 신청해...,"['부전공학점 다 채운다음 이중전공이랑 차이나는 전공점수 채워야되요', '한마디로 ...",NaN


'1.14.0'